## Using pre-trained models for Texture Classification
In this lab, we will use pre-trained models for the same texture classification task. The dataset we will be using is available here: http://dx.doi.org/10.5281/zenodo.53169. 

![alt text](https://www.researchgate.net/profile/Jakob_Kather/publication/303998214/figure/fig7/AS:391073710002224@1470250646407/Representative-images-from-our-dataset-Here-the-first-10-images-of-every-tissue-class.png)

The above figure shows the 8 different classes of tissue we will be trying to identify. 

In [0]:
from __future__ import print_function
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from matplotlib import pylab as plt
from skimage.color import rgb2lab, rgb2hsv

## Step 1: Load pre-trained VGG16 model
We will use the VGG16 model as the feature extractor for this classification task. First, we need to download the pre-trained weights. The model has been trained on the ImageNet image classification dataset. For more details on this dataset see http://www.image-net.org/. 

In [2]:
model = None
model = VGG16(weights='imagenet', include_top=False, input_shape = (150, 150, 3))
#model.summary()

58892288/58889256 [==============================] - 1s 0us/step


## VGG16
Let's examine the VGG16 model. The model has 16 layers as seen above. The model has the following specifications
* Expected input size : 224x224 pixels
* Number of output classes : 1000

Our images are 150x150 pixels in size and come from only 10 categories. In order to use this model for our classification task, we need to do the following:
* Resize images : Our input images can be resized to the appropriate dimensions. Alternatively, we can pad our images to the expected dimensions. Padding leads to additional choices - Do we pad with zeros, duplicate edge pixels or mirror the image across edges ?
* Change the prediction layer : Remove the existing prediction layer and add a new layer that can predict 8 classes.
* Train : Finally, we need to train the network on our data

# Step 2
Load the data and resize images appropriately. Use the helper function in utils.py. The assumption is that you have already cloned the Week3 repo and know the path to the data folder.

In [0]:
def load_crc_data(data_dir):
    y = np.load(os.path.join(data_dir, 'rgb01.npz'))
    labels = y['labels']
    data = y['rgb_data']
    label_str = y['label_str']
    label_str = label_str.tolist() # this is to convert label_str back to a dictionary
    y = []

    for ii in range(2,6):
        filename = os.path.join(data_dir, 'rgb0' + str(ii) + '.npz')
        print('loading ', filename)
        y = np.load(filename)
        labels = np.append(labels, y['labels'], axis=0)
        data = np.append(data, y['rgb_data'], axis=0)
        print(data.shape)

    return data, labels, label_str

In [26]:
! git clone https://github.com/BeaverWorksMedlytics/Week3_public.git
#os.chdir('..')
from skimage.transform import resize
from sklearn.model_selection import train_test_split
data_dir = os.path.join( os.getcwd(), 'Week3_public', 'data', 'crc')
data, labels, label_str = load_crc_data(data_dir)
num_images = data.shape[0]

fatal: destination path 'Week3_public' already exists and is not an empty directory.
loading  /Week3_public/data/crc/rgb02.npz
(2000, 150, 150, 3)
loading  /Week3_public/data/crc/rgb03.npz
(3000, 150, 150, 3)
loading  /Week3_public/data/crc/rgb04.npz
(4000, 150, 150, 3)
loading  /Week3_public/data/crc/rgb05.npz
(5000, 150, 150, 3)


In [0]:
# Resize data
#resized_data = np.zeros((num_images, 224, 224, 3))
#for ii in range(0, num_images):
#    resized_data[ii,::] = resize(data[ii,::], (224, 224, 3), mode='symmetric')
#data = resized_data
#resized_data = []
#print(data.shape)

In [0]:
#with open('resized.npz', 'wb') as fp:
#    np.savez(fp, data=data, labels=labels)

In [0]:
# Split data into train and test sets
train_images, test_images, train_labels, test_labels = train_test_split(data, labels, test_size=.2)
data = None
ntrain = train_images.shape[0]
ntest  = test_images.shape[0]

## Step 3
We will now modify the pre-trained VGG network as described earlier. We will remove the following layers : 

<img src="images/final_layers.png" width="65%"></img> 


In [29]:
# remove the final layer in the model
for ii in range(0,4):
    model.layers.pop()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [30]:
# Convert into sequential
seq_model = None
seq_model = Sequential()
for layer in model.layers:
    seq_model.add( layer )
seq_model.summary()

for ii in range(0,1):
    seq_model.pop()

seq_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 37, 37, 256)       295168    
__________

# Step 4
Freeze the layers that remain in our model.

In [31]:
for layer in seq_model.layers:
    layer.trainable = False
seq_model.summary()

seq_model.add( Flatten(name="my_flatten") )
#seq_model.add( Dense(1024, activation="relu", name="new_dense1") )
#seq_model.add( Dropout(0.5) )
#seq_model.add( Dense(64, activation="relu", name="new_dense2") )
seq_model.add( Dense(8, activation="relu", name="predictions") )

seq_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 37, 37, 256)       295168    
__________

In [0]:
seq_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics = ['accuracy'])

In [0]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [0]:
y = seq_model.fit(train_images, train_labels, validation_data=(test_images, test_labels),
                  batch_size=64, epochs=100, verbose=1)

In [21]:
print(y.history.keys())
# summarize history for accuracy
plt.plot(y.history['acc'])
plt.plot(y.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(y.history['loss'])
plt.plot(y.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: ignored

In [22]:
# convert images to LAB space and normalize

lab_train = np.zeros((ntrain, 224, 224, 3))
ii = 0
for im in train_images:
    lab_train[ii,::] = rgb2lab(im)
    ii = ii + 1
    # normalize ?

ValueError: ignored

In [0]:
k = 1000
plt.figure(figsize=(40,20))
plt.subplot(2,4,1)
plt.imshow(train_images[k,::])
plt.axis('off')
for ii in range(0,3):
    plt.subplot(2,4,ii+2)
    plt.imshow(lab_train[k,:,:,ii])
    plt.gray()
    plt.axis('off')
im = train_images[k,::]
hsvim = rgb2hsv(im)
plt.subplot(2,4,6)
plt.imshow(hsvim[:,:,0])
plt.axis('off')
plt.subplot(2,4,7)
plt.imshow(hsvim[:,:,1])
plt.axis('off')
plt.subplot(2,4,8)
plt.imshow(hsvim[:,:,2])
plt.axis('off')
plt.show()